In [1]:
import os
import sys
import re
import pandas as pd

In [2]:
pattern = r"(?<=output_).[^_]*(?=_)"
news_list = set()
files = [f for f in os.listdir() if f.endswith(".csv")]

for f in files:
    news_name = re.search(pattern, f)[0]
    news_list.add(news_name)
    print(f)
        

output_ABC7_0_gcp.csv
output_ABC7_1_gcp.csv
output_ABC7_2_gcp.csv
output_ABCNews_0_gcp.csv
output_ABCNews_0_nyu.csv
output_ABCNews_1_nyu.csv
output_ABCNews_2_nyu.csv
output_ABCNews_3_nyu.csv
output_ABCNews_4_nyu.csv
output_ABCNews_5_nyu.csv
output_ABCNews_full_nyu.csv
output_ABCNews_nyu.csv
output_AssociatedPress_0_gcp.csv
output_AssociatedPress_1_gcp.csv
output_AssociatedPress_2_gcp.csv
output_BBCNews_0_local.csv
output_BBCNews_0_nyu.csv
output_BBCNews_1_nyu.csv
output_BBCNews_2_nyu.csv
output_BBCNews_3_nyu.csv
output_BBCNews_4_nyu.csv
output_BBCNews_nyu.csv
output_BloombergTelevision_0_local.csv
output_BloombergTelevision_0_nyu.csv
output_BloombergTelevision_1_local.csv
output_BloombergTelevision_1_nyu.csv
output_BloombergTelevision_2_local.csv
output_BloombergTelevision_2_nyu.csv
output_BloombergTelevision_3_nyu.csv
output_BloombergTelevision_4_nyu.csv
output_Breibart_0_gcp.csv
output_Breibart_1_gcp.csv
output_Breibart_2_gcp.csv
output_BusinessInsider_0_gcp.csv
output_BusinessInside

In [3]:
print(len(news_list))
news_list = list(news_list)
news_list[0]

36


'NewsNation'

In [4]:
all_df_list = []
for channel in news_list:
    channel_files = [f for f in files if f.startswith(f"output_{channel}")]
    channel_df_list = []

    for f in channel_files:
        df_temp = pd.read_csv(f, encoding= 'unicode_escape')
        channel_df_list.append(df_temp)

    df = pd.concat(channel_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
    all_df_list.append(df)
    
df = pd.DataFrame()
df = pd.concat(all_df_list).drop_duplicates(subset=["video_id"], keep='first').reset_index(drop=True)
df

,video_id,channel_name,video_title
0,UlU-sdGz_cw,NewsNation,Family reports giant non-humans in backyard af...
1,eBE3kZOnpAQ,NewsNation,Vicky White's death 'doesn't surprise' Lauderd...
2,OmeZzJCK1NI,NewsNation,Vicky White's double life revealed | NewsNatio...
3,rHVpZIO03_Y,NewsNation,Exclusive: Former associate says famous names ...
4,ZuhOgXkowro,NewsNation,Elite Canadian sniper joins Ukraine in battle ...
...,...,...,...
33427,IhUhf1TiC3A,TheGuardian,Sending Aya Back: the Syrian teen facing depor...
33428,f62lfHTJuxU,TheGuardian,My favourite chippie - Scarborough's Harbour V...
33429,w5XQ3oHY09s,TheGuardian,How firearm-loving Maine could defeat the gun ...
33430,ygSxeal_Z7g,TheGuardian,Granada's Alhambra palace


In [7]:
df.to_csv("news_videos.csv", index=False, encoding="utf-8")